In [1]:
from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from mkdtree import kdtree

print('START')
MAX_NODES = 9999
num_nodes = 10

node_id_list = [int(MAX_NODES / 2)]
#
tree = kdtree.create_root(4, 40)
print(tree.sel_axis)

START
<function create_root.<locals>.<lambda> at 0x000001D93D13EB80>


In [2]:
print('PRE-LOOP')
while True:
    print('IN-LOOP')
    selected_node = random.randint(0, MAX_NODES)
    if selected_node not in node_id_list:
        node_id_list.append(selected_node)
    if len(node_id_list) == num_nodes:
        break
print('POST-LOOP')

print(node_id_list)

PRE-LOOP
IN-LOOP
IN-LOOP
IN-LOOP
IN-LOOP
IN-LOOP
IN-LOOP
IN-LOOP
IN-LOOP
IN-LOOP
POST-LOOP
[4999, 5325, 50, 3264, 617, 4789, 2069, 2257, 9111, 9751]


In [3]:
print(f'node_id_list size: {len(node_id_list)}')
print(f'node_id_list: {node_id_list}')

# key_pair already exists in Wallet.py
wallets = {}
for n in node_id_list:
    wallets[n] = Wallet()
    # key_pairs[n] = RSA.generate(2048)


node_id_list size: 10
node_id_list: [4999, 5325, 50, 3264, 617, 4789, 2069, 2257, 9111, 9751]


In [4]:
FORGE_POINT = 3

chain1 = MKDBlockchain()
chain2 = MKDBlockchain()

pool1 = TransactionPool()
pool2 = TransactionPool()


def create_transaction(sender, receiver, pool, amount, type):
    transaction = Transaction(sender, receiver, amount, type)
    if not pool.transaction_exists(transaction):
        pool.add_transaction(transaction)

def create_transaction_batch(pool, t_type, t_num):
    if pool1.forging_required():
        forger = list(wallets.values())[(random.randint(0,1000)%10)]
        chain1.add_block(chain1.create_block(pool1.transactions, forger))
        pool1.remove_from_pool(pool1.transactions)
    for i in range(0,t_num):
        # create_transaction(sender, receiver, pool, random.randint(0, 500), t_type)
        sender = list(wallets.values())[(random.randint(0,1000)%10)].public_key_string()
        receiver = list(wallets.values())[(random.randint(0,1000)%10)].public_key_string()

        create_transaction(sender, receiver, pool, random.randint(0, 500), t_type)


create_transaction_batch(pool1, 'EXCHANGE', 1000)
create_transaction_batch(pool2, 'EXCHANGE', 20000)

create_transaction(list(wallets.values())[random.randint(0,1000)%4].public_key_string(),
                   list(wallets.values())[random.randint(0,1000)%4].public_key_string(),
                   pool2, 226, 'TRANSFER')

ts = [t.to_json() for t in pool1.transactions]

# pprint.pprint(ts)
# pprint.pprint(pool1.transactions)
pprint.pprint(chain1.to_json())


print(chain1)
print('END')

ValueError: invalid literal for int() with base 10: '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAzQpbympVpfK3908FgvJ7\nlFiwOKQFUb/dPL4t7rcuS5ztMSeH82+5IKWSo8AgIxDtAB58fhk9nCc4X3gZsDXG\nv4d8UR0ibdORIjdqUzePc0aeyxr/VANdfUK8/8M

In [ ]:
# then = time.time_ns()
# for i in range(0,100):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'100 iterations: {now}')
#
# then = time.time_ns()
# for i in range(0,1000):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'1000 iterations: {now}')
#
# then = time.time_ns()
# for i in range(0,100000):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'100000 iterations: {now}')
#
# then = time.time_ns()
# for i in range(0,1000000):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'1000000 iterations: {now}')
#
# then = time.time_ns()
# for i in range(0,1000000000):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'100000000 iterations: {now}')
#
# then = time.time_ns()
# for i in range(0,1000000000):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'1000000000 iterations: {now}')
#
# then = time.time_ns()
# for i in range(0,1000000000):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'10000000000 iterations: {now}')
#
# then = time.time_ns()
# for i in range(0,1000000000):
#     block.Block([], '0', i, random.randint(0, 1000), random.randint(0, 1000), 0, i)
# now = time.time_ns() - then
# print(f'100000000000 iterations: {now}')

In [ ]:
# def move_listener(listener, oldtopic, newtopic):
#     pub.unsubscribe(listener, oldtopic) #core.TopicManager.getTopicsSubscribed(listener))
#     pub.subscribe(listener, newtopic)
#
# with open('data.txt') as f:
#     lines = f.readlines() # list containing lines of file
#     i = 0
#     node = []
#     for line in lines:
#         parts = line.split(',')
#         if i < 80:
#             newnode = Node(i + 1, 0)
#             newnode.startListener("c" + str(parts[5]).strip())
#             node.append(newnode)
#         else:
#             move_listener(node[i % 80].node_listener, "c" + str(parts[6]).strip(), "c" + str(parts[5]).strip())
#         i += 1